<a href="https://colab.research.google.com/github/dwiputri-git/employee-promotion-ml-project/blob/main/notebooks/02_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import library
import warnings
import pandas as pd
import numpy as np

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# Load data
df = pd.read_csv('Rakamin Bootcamp - Dataset - Promotion Dataset.csv', sep = ';')

In [ ]:
# 1. Cek missing value

df.isnull().sum()

,0
Employee_ID,0
Age,50
Years_at_Company,49
Performance_Score,50
Leadership_Score,50
Training_Hours,50
Projects_Handled,50
Peer_Review_Score,50
Current_Position_Level,50
Promotion_Eligible,50


In [ ]:
df_clean = df.copy()

In [ ]:
# 2. Handling Missing Values (1)
# Hapus baris dengan missing values Promotion_Eligible (target)
df_clean = df_clean.dropna(subset=['Promotion_Eligible'])

In [ ]:
# 3. Ubah tipe data Promotion_Eligible
df_clean['Promotion_Eligible'] = df_clean['Promotion_Eligible'].astype(int)

In [ ]:
# 4. Handling Missing Values (2)

# --- Kolom numerik ---
num_cols = df_clean.select_dtypes(include=[np.number]).columns.drop(['Promotion_Eligible'])

# Cek skewness setiap kolom numerik
for col in num_cols:
    skewness = df_clean[col].skew()
    print(f"{col} → skewness = {skewness:.2f}")

    if abs(skewness) > 1:
        # Skewed → missing values isi dengan median
        df_clean[col].fillna(df_clean[col].median(), inplace=True)
        print(f"   • Skewed → Imputasi oleh median ({df_clean[col].median():.2f})")
    else:
        # Relatif normal → missing values isi dengan mean
        df_clean[col].fillna(df_clean[col].mean(), inplace=True)
        print(f"   • Normal → Imputasi oleh mean ({df_clean[col].mean():.2f})")

print("\nMissing value handled!")


Age → skewness = 2.70
   • Skewed → Imputasi oleh median (41.00)
Years_at_Company → skewness = 0.88
   • Normal → Imputasi oleh mean (14.90)
Performance_Score → skewness = 0.01
   • Normal → Imputasi oleh mean (2.96)
Leadership_Score → skewness = -0.04
   • Normal → Imputasi oleh mean (50.16)
Training_Hours → skewness = 20.62
   • Skewed → Imputasi oleh median (103.00)
Projects_Handled → skewness = -0.07
   • Normal → Imputasi oleh mean (9.96)
Peer_Review_Score → skewness = -0.06
   • Normal → Imputasi oleh mean (51.12)

Missing value handled!


In [ ]:
cat_cols = df_clean.select_dtypes(['object']).columns
cat_cols

Index(['Employee_ID', 'Current_Position_Level'], dtype='object')

In [ ]:
# --- Kategorikal ---
col = 'Current_Position_Level'

# Cek apakah ada missing values
if df_clean[col].isna().sum() > 0:
    impute_value = df_clean[col].mode()[0]
    df_clean[col].fillna(impute_value, inplace=True)
    print(f"{col}: Imputasi oleh modus ('{impute_value}')")

Current_Position_Level: Imputasi oleh modus ('Mid')


In [ ]:
# 5. Handling Duplicate
duplicate_count = df_clean.duplicated().sum()
print(f"Jumlah baris duplikat: {duplicate_count}")

# Jika ada duplikat → hapus
if duplicate_count > 0:
    df_clean = df_clean.drop_duplicates()
    print(f"Setelah dihapus, ukuran data: {df_clean.shape}")


Jumlah baris duplikat: 0


In [ ]:
# 6. Handling Outlier

# Deteksi outlier dengan metode IQR
for col in num_cols:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    mask_outlier = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
    outlier_pct = 100 * mask_outlier.sum() / len(df_clean)

    if outlier_pct < 5:
        # Sedikit outlier → hapus barisnya
        df_clean = df_clean[~mask_outlier]
        print(f"{col}: {outlier_pct:.2f}% outlier → dihapus ({mask_outlier.sum()} baris)")
    else:
        # Banyak outlier → tangani dengan winsorization
        df_clean[col] = np.where(df_clean[col] < lower_bound, lower_bound,
                  np.where(df_clean[col] > upper_bound, upper_bound, df_clean[col]))
        print(f"{col}: {outlier_pct:.2f}% outlier → dihandling (winsorization)")


Age: 0.42% outlier → dihapus (4 baris)
Years_at_Company: 0.21% outlier → dihapus (2 baris)
Performance_Score: 0.00% outlier → dihapus (0 baris)
Leadership_Score: 0.00% outlier → dihapus (0 baris)
Training_Hours: 0.42% outlier → dihapus (4 baris)
Projects_Handled: 0.00% outlier → dihapus (0 baris)
Peer_Review_Score: 0.00% outlier → dihapus (0 baris)


In [ ]:
# 7. Deteksi & Hapus Nilai Negatif

# Deteksi nilai negatif (anomali) di setiap kolom numerik
neg_anomalies = {}

for col in num_cols:
    neg_count = (df_clean[col] < 0).sum()
    if neg_count > 0:
        neg_anomalies[col] = neg_count
        print(f"{col}: {neg_count} nilai negatif ditemukan")

# Jika ada nilai negatif → hapus barisnya
if len(neg_anomalies) > 0:
    total_before = df_clean.shape[0]
    df_clean = df_clean[(df_clean[num_cols] >= 0).all(axis=1)]
    total_after = df_clean.shape[0]
    print(f"Jumlah baris dihapus: {total_before - total_after}")
else:
    print("Tidak ada nilai negatif yang ditemukan")

Years_at_Company: 1 nilai negatif ditemukan
Jumlah baris dihapus: 1


In [ ]:
# 8. Simpan Hasil Data Cleaning
df_clean.to_csv('Promotion Dataset_Clean.csv', index=False)
print("Data bersih : **Promotion Dataset_Clean.csv** disimpan.")


Data bersih : **Promotion Dataset_Clean.csv** disimpan.
